In [1]:
# If running in a notebook for the first time:
%pip install openai python-dotenv ipywidgets 
# --quiet


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# --- IMPORTS ---
import os
import json
import time
import openai
from dotenv import load_dotenv
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown

# --- LOAD API KEY ---
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("❌ OPENAI_API_KEY not found in .env file!")
openai.api_key = api_key

# --- GAME STATE ---
model_name = "gpt-3.5-turbo"
player_name = "Ihno"
context = ""
game_memory = []
player_stats = {}
inventory = []
difficulty = 1

# --- DIFFICULTY ADJUSTMENT ---
def adjust_difficulty(player_input):
    global difficulty
    if any(word in player_input.lower() for word in ["attack", "fight", "battle"]):
        difficulty = min(difficulty + 1, 5)
    elif any(word in player_input.lower() for word in ["run", "talk", "hide"]):
        difficulty = max(difficulty - 1, 1)
    return difficulty

# --- STORY GENERATION --- (Updated)
def generate_story(context, player_input, difficulty):
    prompt = (
        "You are a fantasy storytelling AI. "
        "Continue the adventure in a vivid, immersive style. "
        f"Difficulty level: {difficulty}.\n\n"
        f"{context}\n"
        f"{player_name}: {player_input}\n"
        "Narrator:"
    )
    
    for attempt in range(5):
        try:
            # Use openai.Completion.create for OpenAI API >= 1.0.0
            response = openai.Completion.create(
                model=model_name,
                prompt=prompt,
                max_tokens=150,
                temperature=0.9,
                top_p=0.95
            )
            return response.choices[0].text.strip()  # Updated to use .text for the response
        except openai.error.RateLimitError:
            time.sleep(2 ** attempt)  # Exponential backoff for rate limiting
        except Exception as e:
            return f"❌ Error generating story: {e}"
    return "⚠️ The story could not be continued due to repeated errors."

# --- PRINT GAME STATE ---
def print_game_state():
    display(Markdown(f"### 📖 **Story so far**\n{context}"))
    display(Markdown(f"**🧍 {player_name}'s Inventory:** {inventory}"))
    display(Markdown(f"**❤️ Stats:** {player_stats} | 🎯 Difficulty:** {['Easy', 'Medium', 'Hard', 'Very Hard', 'Nightmare'][difficulty - 1]}"))

# --- PLAY TURN ---
def play_turn(player_input):
    global context
    if not player_input.strip():
        return
    game_memory.append(f"{player_name}: {player_input}")
    adjust_difficulty(player_input)

    recent_context = "\n".join(game_memory[-6:])
    new_story = generate_story(recent_context, player_input, difficulty)
    context_update = f"\n{player_name}: {player_input}\n{new_story}"
    context += context_update
    game_memory.append(new_story)

    output_area.clear_output(wait=True)
    with output_area:
        print_game_state()

# --- START GAME ---
def start_new_game():
    global context, player_stats, inventory, difficulty, game_memory
    context = f"{player_name} awakens in a dark forest. A mysterious figure approaches."
    game_memory = [context]
    player_stats = {"health": 100, "strength": 10, "gold": 5}
    inventory = ["torch"]
    difficulty = 1
    output_area.clear_output()
    with output_area:
        display(Markdown("🆕 **New game started.**"))
        print_game_state()

# --- UI SETUP ---
input_box = widgets.Text(
    placeholder='What does Ihno do next?',
    description='▶️ Action:',
    layout=widgets.Layout(width='70%')
)
submit_button = widgets.Button(description="Submit", button_style='success')

output_area = widgets.Output()

def on_button_click(b):
    play_turn(input_box.value)
    input_box.value = ''

submit_button.on_click(on_button_click)

# Display all widgets
start_new_game()
display(input_box, submit_button, output_area)


Text(value='', description='▶️ Action:', layout=Layout(width='70%'), placeholder='What does Ihno do next?')

Button(button_style='success', description='Submit', style=ButtonStyle())

Output()

AttributeError: module 'openai' has no attribute 'error'

Probeersel 2

In [6]:
%pip install --upgrade openai

   ---------------------------------------- 0.0/662.0 kB ? eta -:--:--
   --------------------------------------- 662.0/662.0 kB 12.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.74.0
    Uninstalling openai-1.74.0:
      Successfully uninstalled openai-1.74.0
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
# --- IMPORTS ---
import os
import json
import time
import openai
from dotenv import load_dotenv
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown

# --- LOAD API KEY ---
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("❌ OPENAI_API_KEY not found in .env file!")
openai.api_key = api_key

# --- GAME STATE ---
model_name = "gpt-3.5-turbo"
player_name = "Ihno"
context = ""
game_memory = []
player_stats = {}
inventory = []
difficulty = 1

# --- DIFFICULTY ADJUSTMENT ---
def adjust_difficulty(player_input):
    global difficulty
    if any(word in player_input.lower() for word in ["attack", "fight", "battle"]):
        difficulty = min(difficulty + 1, 5)
    elif any(word in player_input.lower() for word in ["run", "talk", "hide"]):
        difficulty = max(difficulty - 1, 1)
    return difficulty

# --- STORY GENERATION --- (Updated for v1.0.0)
def generate_story(context, player_input, difficulty):
    prompt = (
        "You are a fantasy storytelling AI. "
        "Continue the adventure in a vivid, immersive style. "
        f"Difficulty level: {difficulty}.\n\n"
        f"{context}\n"
        f"{player_name}: {player_input}\n"
        "Narrator:"
    )
    
    for attempt in range(5):
        try:
            # Use the new `openai.Chat.create` for the updated API
            response = openai.Chat.create(
                model=model_name,
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=150,
                temperature=0.9,
                top_p=0.95
            )
            return response["choices"][0]["message"]["content"].strip()  # Updated for new API format
        except openai.error.RateLimitError:
            time.sleep(2 ** attempt)  # Exponential backoff for rate limiting
        except Exception as e:
            return f"❌ Error generating story: {e}"
    return "⚠️ The story could not be continued due to repeated errors."

# --- PRINT GAME STATE ---
def print_game_state():
    display(Markdown(f"### 📖 **Story so far**\n{context}"))
    display(Markdown(f"**🧍 {player_name}'s Inventory:** {inventory}"))
    display(Markdown(f"**❤️ Stats:** {player_stats} | 🎯 Difficulty:** {['Easy', 'Medium', 'Hard', 'Very Hard', 'Nightmare'][difficulty - 1]}"))

# --- PLAY TURN ---
def play_turn(player_input):
    global context
    if not player_input.strip():
        return
    game_memory.append(f"{player_name}: {player_input}")
    adjust_difficulty(player_input)

    recent_context = "\n".join(game_memory[-6:])
    new_story = generate_story(recent_context, player_input, difficulty)
    context_update = f"\n{player_name}: {player_input}\n{new_story}"
    context += context_update
    game_memory.append(new_story)

    output_area.clear_output(wait=True)
    with output_area:
        print_game_state()

# --- START GAME ---
def start_new_game():
    global context, player_stats, inventory, difficulty, game_memory
    context = f"{player_name} awakens in a dark forest. A mysterious figure approaches."
    game_memory = [context]
    player_stats = {"health": 100, "strength": 10, "gold": 5}
    inventory = ["torch"]
    difficulty = 1
    output_area.clear_output()
    with output_area:
        display(Markdown("🆕 **New game started.**"))
        print_game_state()

# --- UI SETUP ---
input_box = widgets.Text(
    placeholder='What does Ihno do next?',
    description='▶️ Action:',
    layout=widgets.Layout(width='70%')
)
submit_button = widgets.Button(description="Submit", button_style='success')

output_area = widgets.Output()

def on_button_click(b):
    play_turn(input_box.value)
    input_box.value = ''

submit_button.on_click(on_button_click)

# Display all widgets
start_new_game()
display(input_box, submit_button, output_area)


Text(value='', description='▶️ Action:', layout=Layout(width='70%'), placeholder='What does Ihno do next?')

Button(button_style='success', description='Submit', style=ButtonStyle())

Output()

AttributeError: module 'openai' has no attribute 'error'

Probeersel 3

In [17]:
# --- IMPORTS ---
import os
import time
from openai import OpenAI, OpenAIError, RateLimitError
from dotenv import load_dotenv
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown

# --- LOAD API KEY ---
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("❌ OPENAI_API_KEY not found in .env file!")

client = OpenAI(api_key=api_key)

# --- GAME STATE ---
model_name = "gpt-3.5-turbo"
player_name = "Ihno"
context = ""
game_memory = []
player_stats = {}
inventory = []
difficulty = 1

# --- DIFFICULTY ADJUSTMENT ---
def adjust_difficulty(player_input):
    global difficulty
    if any(word in player_input.lower() for word in ["attack", "fight", "battle"]):
        difficulty = min(difficulty + 1, 5)
    elif any(word in player_input.lower() for word in ["run", "talk", "hide"]):
        difficulty = max(difficulty - 1, 1)
    return difficulty

# --- STORY GENERATION (OpenAI SDK v1) ---
def generate_story(context, player_input, difficulty):
    prompt = (
        "You are a fantasy storytelling AI. "
        "Continue the adventure in a vivid, immersive style. "
        f"Difficulty level: {difficulty}.\n\n"
        f"{context}\n"
        f"{player_name}: {player_input}\n"
        "Narrator:"
    )

    for attempt in range(5):
        try:
            response = client.chat.completions.create(
                model=model_name,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=150,
                temperature=0.9,
                top_p=0.95
            )
            return response.choices[0].message.content.strip()
        except RateLimitError:
            time.sleep(2 ** attempt)
        except OpenAIError as e:
            return f"❌ Error generating story: {e}"

    return "⚠️ The story could not be continued due to repeated errors."

# --- PRINT GAME STATE ---
def print_game_state():
    display(Markdown(f"### 📖 **Story so far**\n{context}"))
    display(Markdown(f"**🧍 {player_name}'s Inventory:** {inventory}"))
    display(Markdown(f"**❤️ Stats:** {player_stats} | 🎯 Difficulty:** {['Easy', 'Medium', 'Hard', 'Very Hard', 'Nightmare'][difficulty - 1]}"))

# --- PLAY TURN ---
def play_turn(player_input):
    global context
    if not player_input.strip():
        return
    game_memory.append(f"{player_name}: {player_input}")
    adjust_difficulty(player_input)

    recent_context = "\n".join(game_memory[-6:])
    new_story = generate_story(recent_context, player_input, difficulty)
    context_update = f"\n{player_name}: {player_input}\n{new_story}"
    context += context_update
    game_memory.append(new_story)

    output_area.clear_output(wait=True)
    with output_area:
        print_game_state()

# --- START GAME ---
def start_new_game():
    global context, player_stats, inventory, difficulty, game_memory
    context = f"{player_name} awakens in a dark forest. A mysterious figure approaches."
    game_memory = [context]
    player_stats = {"health": 100, "strength": 10, "gold": 5}
    inventory = ["torch"]
    difficulty = 1
    output_area.clear_output()
    with output_area:
        display(Markdown("🆕 **New game started.**"))
        print_game_state()

# --- UI SETUP ---
input_box = widgets.Text(
    placeholder='What does Ihno do next?',
    description='▶️ Action:',
    layout=widgets.Layout(width='70%')
)
submit_button = widgets.Button(description="Submit", button_style='success')
output_area = widgets.Output()

def on_button_click(b):
    play_turn(input_box.value)
    input_box.value = ''

submit_button.on_click(on_button_click)

# --- START & DISPLAY ---
start_new_game()
display(input_box, submit_button, output_area)



Text(value='', description='▶️ Action:', layout=Layout(width='70%'), placeholder='What does Ihno do next?')

Button(button_style='success', description='Submit', style=ButtonStyle())

Output()